In [1]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, Matern
import matplotlib.pyplot as plt

In [2]:
dataset = np.load("processed_dataset.npz")
X_train = dataset['X_train']
Y_train = dataset['Y_train']
X_test = dataset['X_test']
Y_test = dataset['Y_test']

scaler = MinMaxScaler()
Y_train = scaler.fit_transform(Y_train)
Y_test = scaler.transform(Y_test)

# Convert to classification labels by checking if Y[:2] is greater than 0.5
Y_train = (Y_train[:, 2] > 0.5).astype(int)
Y_test = (Y_test[:, 2] > 0.5).astype(int)

In [ ]:
kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 10.0)) + WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))
gp = GaussianProcessClassifier(kernel=kernel, n_restarts_optimizer=10)
gp.fit(X_train, Y_train)
print(gp.kernel_)

In [ ]:
Y_train_pred = gp.predict(X_train)
Y_test_pred = gp.predict(X_test)

print("Train accuracy:", np.mean(Y_train_pred == Y_train))
print("Test accuracy:", np.mean(Y_test_pred == Y_test))

In [ ]:
# Interpolate between first and last test set points

def create_interpolated_test_set(X_test, num_interpolations):
    first = X_test[0]
    last = X_test[-1]

    interpolated_test_set = np.zeros((num_interpolations, X_test.shape[1]))
    for i in range(num_interpolations):
        interpolated_test_set[i] = first + (last - first) * i / num_interpolations
    return interpolated_test_set

interpolated_test_set = create_interpolated_test_set(X_test, 100)


In [ ]:
y_pred_interp = gp.predict(interpolated_test_set)

plt.plot(y_pred_interp)